# Email Information Extraction with DSPy

This notebook demonstrates how to build intelligent email processing and information extraction systems using DSPy.

## What You'll Learn:
- Email parsing and preprocessing
- Information extraction from email content
- Classification of email types and priorities
- Action item identification and task generation
- Contact and entity extraction
- Sentiment analysis for customer emails

Based on the DSPy tutorial: [Email Information Extraction](https://dspy.ai/tutorials/email_extraction/)

## Setup and Imports

In [ ]:
import os
import sys
sys.path.append('../../')

import dspy
import re
import email
import json
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime, timedelta
from dataclasses import dataclass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from utils import setup_default_lm, print_step, print_result, print_error
from dotenv import load_dotenv

# Load environment variables
load_dotenv('../../.env')

## Language Model Configuration

In [ ]:
print_step("Setting up Language Model", "Configuring DSPy for email processing")

try:
    lm = setup_default_lm(provider="openai", model="gpt-4o-mini", max_tokens=2000)
    dspy.configure(lm=lm)
    print_result("Language model configured successfully!", "Status")
except Exception as e:
    print_error(f"Failed to configure language model: {e}")

## Data Structures

Define data structures for email processing and extraction results.

In [ ]:
@dataclass
class EmailData:
    """Structured representation of email data."""
    subject: str
    sender: str
    recipients: List[str]
    body: str
    timestamp: datetime
    attachments: List[str] = None
    
@dataclass
class ExtractedInfo:
    """Information extracted from email."""
    email_type: str
    priority: str
    sentiment: str
    action_items: List[str]
    key_entities: Dict[str, List[str]]
    summary: str
    next_steps: List[str]
    deadline_mentions: List[str]

@dataclass
class ContactInfo:
    """Contact information extracted from email."""
    name: str
    email: str
    phone: Optional[str] = None
    company: Optional[str] = None
    role: Optional[str] = None

# Sample email data for testing
sample_emails = [
    EmailData(
        subject="Project Update - Q4 Launch Timeline",
        sender="sarah.johnson@company.com",
        recipients=["team@company.com"],
        body="""Hi Team,
        
        I wanted to provide an update on our Q4 product launch timeline. Based on the latest development progress, we're on track to complete beta testing by November 15th.
        
        Key milestones:
        - Complete user testing: November 10th
        - Fix critical bugs: November 12th 
        - Final QA review: November 15th
        - Marketing campaign launch: November 20th
        - Public release: December 1st
        
        Please let me know if you have any concerns about these dates. We need to finalize the marketing materials by next Friday.
        
        Best regards,
        Sarah Johnson
        Product Manager
        sarah.johnson@company.com
        (555) 123-4567
        """,
        timestamp=datetime.now() - timedelta(days=1)
    ),
    
    EmailData(
        subject="URGENT: Server Issues - Need Immediate Action",
        sender="alerts@monitoring.com",
        recipients=["devops@company.com", "management@company.com"],
        body="""CRITICAL ALERT
        
        Server cluster prod-web-01 is experiencing high CPU utilization (95%+) and memory issues.
        
        Error details:
        - Memory usage: 98% of 32GB
        - CPU usage: 96% average over last 15 minutes
        - Active connections: 15,000+ (normal: 5,000)
        - Response time: 8.5s (normal: 0.3s)
        
        IMMEDIATE ACTIONS REQUIRED:
        1. Scale up additional server instances
        2. Investigate database query performance
        3. Check for potential DDoS attack
        4. Contact John Smith (DevOps Lead) at john.smith@company.com or (555) 987-6543
        
        This issue is affecting customer experience. Please address within the next 30 minutes.
        
        Monitoring System
        """,
        timestamp=datetime.now() - timedelta(hours=2)
    ),
    
    EmailData(
        subject="Thank you for the amazing service!",
        sender="happy.customer@email.com",
        recipients=["support@company.com"],
        body="""Dear Support Team,
        
        I just wanted to reach out and thank you for the exceptional service I received yesterday. 
        Lisa from your customer success team went above and beyond to help me resolve the billing issue.
        
        Your response time was incredibly fast (less than 2 hours) and the solution was perfect. 
        I've been a customer for 3 years and this level of service is why I continue to recommend 
        your company to my colleagues.
        
        Please pass along my thanks to Lisa and the entire support team.
        
        Best wishes,
        Michael Chen
        CEO, TechStart Inc.
        michael.chen@techstart.com
        """,
        timestamp=datetime.now() - timedelta(hours=6)
    )
]

print_result(f"Loaded {len(sample_emails)} sample emails for processing")

## Email Processing Signatures

Define DSPy signatures for various email processing tasks.

In [ ]:
class EmailClassification(dspy.Signature):
    """Classify email type and determine priority level."""
    
    email_subject = dspy.InputField(desc="Subject line of the email")
    email_body = dspy.InputField(desc="Main content of the email")
    sender_info = dspy.InputField(desc="Information about the email sender")
    
    email_type = dspy.OutputField(desc="Type of email: business_update, urgent_alert, customer_feedback, meeting_request, support_ticket, etc.")
    priority = dspy.OutputField(desc="Priority level: HIGH, MEDIUM, LOW")
    urgency_reason = dspy.OutputField(desc="Explanation for the assigned priority level")

class ActionItemExtraction(dspy.Signature):
    """Extract action items and tasks from email content."""
    
    email_content = dspy.InputField(desc="Complete email content including subject and body")
    
    action_items = dspy.OutputField(desc="List of specific action items or tasks mentioned in the email")
    deadlines = dspy.OutputField(desc="Any deadlines or time-sensitive items mentioned")
    responsible_parties = dspy.OutputField(desc="People or teams responsible for actions")
    next_steps = dspy.OutputField(desc="Recommended next steps based on email content")

class EntityExtraction(dspy.Signature):
    """Extract entities and contact information from email."""
    
    email_text = dspy.InputField(desc="Email content to analyze for entities")
    
    people = dspy.OutputField(desc="Names of people mentioned in the email")
    companies = dspy.OutputField(desc="Company or organization names")
    dates = dspy.OutputField(desc="Important dates and deadlines")
    contact_info = dspy.OutputField(desc="Phone numbers, email addresses, or other contact details")
    technical_terms = dspy.OutputField(desc="Technical terms, product names, or project names")

class SentimentAnalysis(dspy.Signature):
    """Analyze sentiment and tone of email communication."""
    
    email_content = dspy.InputField(desc="Email content to analyze for sentiment")
    email_context = dspy.InputField(desc="Context about the email type and purpose")
    
    sentiment = dspy.OutputField(desc="Overall sentiment: POSITIVE, NEGATIVE, NEUTRAL")
    tone = dspy.OutputField(desc="Communication tone: formal, casual, urgent, friendly, frustrated, etc.")
    emotional_indicators = dspy.OutputField(desc="Specific words or phrases indicating emotion")
    customer_satisfaction = dspy.OutputField(desc="If customer email, satisfaction level: satisfied, neutral, dissatisfied")

class EmailSummarization(dspy.Signature):
    """Create a comprehensive summary of email content."""
    
    email_subject = dspy.InputField(desc="Email subject line")
    email_body = dspy.InputField(desc="Email body content")
    extracted_entities = dspy.InputField(desc="Previously extracted entities and action items")
    
    summary = dspy.OutputField(desc="Concise summary of the email's main points")
    key_takeaways = dspy.OutputField(desc="Most important information from the email")
    follow_up_required = dspy.OutputField(desc="Whether follow-up action is needed: yes/no")
    suggested_response = dspy.OutputField(desc="Suggested response or reply if appropriate")

print_result("Email processing signatures defined successfully!")

## Email Processor Module

Create a comprehensive email processing module using DSPy.

In [ ]:
class EmailProcessor(dspy.Module):
    """Comprehensive email processing and information extraction module."""
    
    def __init__(self):
        super().__init__()
        self.classifier = dspy.ChainOfThought(EmailClassification)
        self.action_extractor = dspy.ChainOfThought(ActionItemExtraction)
        self.entity_extractor = dspy.ChainOfThought(EntityExtraction)
        self.sentiment_analyzer = dspy.ChainOfThought(SentimentAnalysis)
        self.summarizer = dspy.ChainOfThought(EmailSummarization)
    
    def preprocess_email(self, email_data: EmailData) -> Dict[str, str]:
        """Preprocess email data for analysis."""
        # Clean and format email content
        clean_body = re.sub(r'\s+', ' ', email_data.body.strip())
        
        # Extract sender domain for context
        sender_domain = email_data.sender.split('@')[-1] if '@' in email_data.sender else ''
        
        return {
            'subject': email_data.subject,
            'body': clean_body,
            'sender': email_data.sender,
            'sender_domain': sender_domain,
            'full_content': f"Subject: {email_data.subject}\n\nFrom: {email_data.sender}\n\n{clean_body}"
        }
    
    def extract_contact_info(self, text: str) -> List[ContactInfo]:
        """Extract contact information using regex patterns."""
        contacts = []
        
        # Extract email addresses
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        emails = re.findall(email_pattern, text)
        
        # Extract phone numbers (simple pattern)
        phone_pattern = r'\(?\d{3}\)?[-\s]?\d{3}[-\s]?\d{4}'
        phones = re.findall(phone_pattern, text)
        
        # Extract names (basic pattern - capitalized words before email/phone)
        name_pattern = r'([A-Z][a-z]+ [A-Z][a-z]+)(?=.*(?:@|\(\d{3}\))?)'
        names = re.findall(name_pattern, text)
        
        # Combine information
        for i, email_addr in enumerate(emails):
            name = names[i] if i < len(names) else ""
            phone = phones[i] if i < len(phones) else None
            contacts.append(ContactInfo(name=name, email=email_addr, phone=phone))
        
        return contacts
    
    def process_email(self, email_data: EmailData) -> ExtractedInfo:
        """Process a single email and extract all relevant information."""
        # Preprocess email
        processed = self.preprocess_email(email_data)
        
        # Step 1: Classify email
        classification = self.classifier(
            email_subject=processed['subject'],
            email_body=processed['body'],
            sender_info=f"Sender: {processed['sender']} (Domain: {processed['sender_domain']})"
        )
        
        # Step 2: Extract action items
        actions = self.action_extractor(
            email_content=processed['full_content']
        )
        
        # Step 3: Extract entities
        entities = self.entity_extractor(
            email_text=processed['full_content']
        )
        
        # Step 4: Analyze sentiment
        sentiment = self.sentiment_analyzer(
            email_content=processed['full_content'],
            email_context=f"Email type: {classification.email_type}, Priority: {classification.priority}"
        )
        
        # Step 5: Generate summary
        summary = self.summarizer(
            email_subject=processed['subject'],
            email_body=processed['body'],
            extracted_entities=f"""Action Items: {actions.action_items}
            People: {entities.people}
            Companies: {entities.companies}
            Dates: {entities.dates}"""
        )
        
        # Extract contacts using regex
        contacts = self.extract_contact_info(processed['full_content'])
        
        # Combine all extracted information
        return ExtractedInfo(
            email_type=classification.email_type,
            priority=classification.priority,
            sentiment=sentiment.sentiment,
            action_items=actions.action_items.split('\n') if actions.action_items else [],
            key_entities={
                'people': entities.people.split(',') if entities.people else [],
                'companies': entities.companies.split(',') if entities.companies else [],
                'dates': entities.dates.split(',') if entities.dates else [],
                'contacts': [f"{c.name} - {c.email}" for c in contacts]
            },
            summary=summary.summary,
            next_steps=actions.next_steps.split('\n') if actions.next_steps else [],
            deadline_mentions=actions.deadlines.split(',') if actions.deadlines else []
        )
    
    def batch_process_emails(self, emails: List[EmailData]) -> List[ExtractedInfo]:
        """Process multiple emails in batch."""
        results = []
        
        for email_data in emails:
            try:
                extracted_info = self.process_email(email_data)
                results.append(extracted_info)
            except Exception as e:
                print_error(f"Error processing email '{email_data.subject}': {e}")
                continue
        
        return results
    
    def generate_report(self, extracted_infos: List[ExtractedInfo]) -> str:
        """Generate a summary report of processed emails."""
        total_emails = len(extracted_infos)
        high_priority = sum(1 for info in extracted_infos if info.priority == 'HIGH')
        action_required = sum(1 for info in extracted_infos if info.action_items)
        
        sentiment_breakdown = {}
        type_breakdown = {}
        
        for info in extracted_infos:
            sentiment_breakdown[info.sentiment] = sentiment_breakdown.get(info.sentiment, 0) + 1
            type_breakdown[info.email_type] = type_breakdown.get(info.email_type, 0) + 1
        
        report = f"""
        EMAIL PROCESSING REPORT
        ======================
        
        Total Emails Processed: {total_emails}
        High Priority Emails: {high_priority}
        Emails Requiring Action: {action_required}
        
        SENTIMENT BREAKDOWN:
        {dict(sentiment_breakdown)}
        
        EMAIL TYPE BREAKDOWN:
        {dict(type_breakdown)}
        """
        
        return report

# Initialize the email processor
processor = EmailProcessor()
print_result("Email processor initialized successfully!")

## Email Processing Demo

Process our sample emails and extract information.

In [ ]:
print_step("Email Processing Demo", "Processing sample emails")

# Process all sample emails
extracted_results = []

for i, email_data in enumerate(sample_emails, 1):
    try:
        print(f"\n{'='*60}")
        print(f"Processing Email {i}: {email_data.subject[:50]}...")
        print('='*60)
        
        extracted_info = processor.process_email(email_data)
        extracted_results.append(extracted_info)
        
        # Display results
        print_result(f"Type: {extracted_info.email_type}", "Email Classification")
        print_result(f"Priority: {extracted_info.priority}", "Priority Level")
        print_result(f"Sentiment: {extracted_info.sentiment}", "Sentiment Analysis")
        print_result(f"Summary: {extracted_info.summary}", "Email Summary")
        
        if extracted_info.action_items:
            print_result("\n".join([f"- {item.strip()}" for item in extracted_info.action_items if item.strip()]), "Action Items")
        
        if extracted_info.deadline_mentions:
            print_result(", ".join(extracted_info.deadline_mentions), "Deadlines")
        
        if extracted_info.key_entities['people']:
            people = [p.strip() for p in extracted_info.key_entities['people'] if p.strip()]
            if people:
                print_result(", ".join(people), "People Mentioned")
        
        if extracted_info.key_entities['contacts']:
            contacts = [c.strip() for c in extracted_info.key_entities['contacts'] if c.strip()]
            if contacts:
                print_result("\n".join(contacts), "Contact Information")
        
    except Exception as e:
        print_error(f"Error processing email {i}: {e}")
        continue

print(f"\n\nSuccessfully processed {len(extracted_results)} emails")

## Batch Processing and Reporting

Demonstrate batch processing capabilities and generate summary reports.

In [ ]:
print_step("Batch Processing", "Processing emails in batch and generating reports")

# Process emails in batch
batch_results = processor.batch_process_emails(sample_emails)

# Generate comprehensive report
report = processor.generate_report(batch_results)
print_result(report, "Processing Report")

# Detailed analysis by priority
print("\n" + "="*60)
print("PRIORITY ANALYSIS")
print("="*60)

high_priority_emails = [info for info in batch_results if info.priority == 'HIGH']
medium_priority_emails = [info for info in batch_results if info.priority == 'MEDIUM']
low_priority_emails = [info for info in batch_results if info.priority == 'LOW']

print_result(f"High Priority: {len(high_priority_emails)} emails", "Priority Breakdown")
for i, info in enumerate(high_priority_emails, 1):
    print(f"  {i}. {info.email_type} - {info.summary[:100]}...")

print_result(f"Medium Priority: {len(medium_priority_emails)} emails", "")
print_result(f"Low Priority: {len(low_priority_emails)} emails", "")

# Action items summary
print("\n" + "="*60)
print("ACTION ITEMS SUMMARY")
print("="*60)

all_action_items = []
for info in batch_results:
    all_action_items.extend(info.action_items)

if all_action_items:
    print_result(f"Total Action Items: {len(all_action_items)}", "Action Summary")
    for i, action in enumerate(all_action_items[:10], 1):  # Show first 10
        if action.strip():
            print(f"  {i}. {action.strip()}")
    
    if len(all_action_items) > 10:
        print(f"  ... and {len(all_action_items) - 10} more action items")
else:
    print_result("No action items found", "Action Summary")

## Advanced Features

Demonstrate advanced email processing features like automated response suggestions and follow-up tracking.

In [ ]:
print_step("Advanced Features", "Automated response suggestions and follow-up tracking")

class ResponseGeneration(dspy.Signature):
    """Generate appropriate email responses based on content and context."""
    
    original_email = dspy.InputField(desc="Original email content and context")
    email_analysis = dspy.InputField(desc="Analysis results including sentiment, priority, and action items")
    response_type = dspy.InputField(desc="Type of response needed: acknowledgment, action_confirmation, information_request, etc.")
    
    response_draft = dspy.OutputField(desc="Professional draft response email")
    tone_recommendation = dspy.OutputField(desc="Recommended tone for the response")
    key_points = dspy.OutputField(desc="Key points that should be addressed in the response")

class FollowUpTracking(dspy.Signature):
    """Identify emails that require follow-up and suggest timing."""
    
    email_content = dspy.InputField(desc="Email content and extracted information")
    action_items = dspy.InputField(desc="Identified action items and deadlines")
    
    requires_followup = dspy.OutputField(desc="Whether follow-up is needed: yes/no")
    followup_timing = dspy.OutputField(desc="When to follow up: immediate, 1_day, 3_days, 1_week, etc.")
    followup_reason = dspy.OutputField(desc="Reason for follow-up")
    followup_content = dspy.OutputField(desc="Suggested follow-up message")

# Create advanced processing modules
response_generator = dspy.ChainOfThought(ResponseGeneration)
followup_tracker = dspy.ChainOfThought(FollowUpTracking)

# Process each email for response suggestions
for i, (email_data, extracted_info) in enumerate(zip(sample_emails, batch_results), 1):
    try:
        print(f"\n{'='*60}")
        print(f"Advanced Processing - Email {i}")
        print('='*60)
        
        # Determine response type based on email type and priority
        if extracted_info.priority == 'HIGH':
            response_type = "urgent_acknowledgment"
        elif 'thank' in email_data.subject.lower() or extracted_info.sentiment == 'POSITIVE':
            response_type = "appreciation_response"
        elif extracted_info.action_items:
            response_type = "action_confirmation"
        else:
            response_type = "general_acknowledgment"
        
        # Generate response suggestion
        response = response_generator(
            original_email=f"Subject: {email_data.subject}\n\nFrom: {email_data.sender}\n\n{email_data.body[:500]}...",
            email_analysis=f"""Type: {extracted_info.email_type}
            Priority: {extracted_info.priority}
            Sentiment: {extracted_info.sentiment}
            Action Items: {', '.join(extracted_info.action_items[:3])}
            Summary: {extracted_info.summary}""",
            response_type=response_type
        )
        
        # Track follow-up requirements
        followup = followup_tracker(
            email_content=extracted_info.summary,
            action_items="\n".join(extracted_info.action_items)
        )
        
        # Display results
        print_result(f"Response Type: {response_type}", "Response Analysis")
        print_result(f"Tone: {response.tone_recommendation}", "Recommended Tone")
        print_result(f"Key Points: {response.key_points}", "Key Points to Address")
        print_result(f"Draft Response:\n{response.response_draft}", "Suggested Response")
        
        print_result(f"Follow-up Required: {followup.requires_followup}", "Follow-up Tracking")
        if followup.requires_followup.lower() == 'yes':
            print_result(f"Timing: {followup.followup_timing}", "Follow-up Timing")
            print_result(f"Reason: {followup.followup_reason}", "Follow-up Reason")
            
    except Exception as e:
        print_error(f"Error in advanced processing for email {i}: {e}")
        continue

## Integration Examples

Show how to integrate email processing with external systems and workflows.

In [ ]:
print_step("Integration Examples", "Connecting with external systems and workflows")

def create_task_from_email(email_data: EmailData, extracted_info: ExtractedInfo) -> Dict[str, Any]:
    """Create a task management system entry from email."""
    task = {
        'title': f"Email: {email_data.subject}",
        'description': extracted_info.summary,
        'priority': extracted_info.priority.lower(),
        'action_items': extracted_info.action_items,
        'due_date': None,  # Would extract from deadline_mentions
        'assigned_to': None,  # Would extract from responsible parties
        'tags': [extracted_info.email_type, extracted_info.sentiment.lower()],
        'source': 'email',
        'created_at': datetime.now().isoformat()
    }
    return task

def create_crm_entry(email_data: EmailData, extracted_info: ExtractedInfo) -> Dict[str, Any]:
    """Create a CRM system entry from customer email."""
    # Extract contact information
    contacts = processor.extract_contact_info(email_data.body)
    
    crm_entry = {
        'contact_email': email_data.sender,
        'interaction_type': extracted_info.email_type,
        'sentiment': extracted_info.sentiment,
        'summary': extracted_info.summary,
        'action_required': len(extracted_info.action_items) > 0,
        'priority': extracted_info.priority,
        'timestamp': email_data.timestamp.isoformat(),
        'extracted_entities': extracted_info.key_entities,
        'follow_up_needed': len(extracted_info.next_steps) > 0
    }
    
    if contacts:
        crm_entry['contact_info'] = {
            'name': contacts[0].name,
            'company': contacts[0].company,
            'phone': contacts[0].phone
        }
    
    return crm_entry

def generate_alert(email_data: EmailData, extracted_info: ExtractedInfo) -> Dict[str, Any]:
    """Generate alert for high-priority emails."""
    if extracted_info.priority == 'HIGH':
        alert = {
            'type': 'high_priority_email',
            'subject': email_data.subject,
            'sender': email_data.sender,
            'summary': extracted_info.summary,
            'action_items': extracted_info.action_items,
            'urgency_reason': f"Email classified as {extracted_info.email_type} with HIGH priority",
            'created_at': datetime.now().isoformat(),
            'requires_immediate_attention': True
        }
        return alert
    return None

# Demonstrate integrations
integration_results = {
    'tasks_created': [],
    'crm_entries': [],
    'alerts_generated': []
}

for email_data, extracted_info in zip(sample_emails, batch_results):
    # Create task if action items exist
    if extracted_info.action_items:
        task = create_task_from_email(email_data, extracted_info)
        integration_results['tasks_created'].append(task)
    
    # Create CRM entry for customer communications
    if 'customer' in extracted_info.email_type.lower() or 'support' in extracted_info.email_type.lower():
        crm_entry = create_crm_entry(email_data, extracted_info)
        integration_results['crm_entries'].append(crm_entry)
    
    # Generate alert for high-priority emails
    alert = generate_alert(email_data, extracted_info)
    if alert:
        integration_results['alerts_generated'].append(alert)

# Display integration results
print_result(f"Tasks Created: {len(integration_results['tasks_created'])}", "Task Management Integration")
for task in integration_results['tasks_created']:
    print(f"  - {task['title']} (Priority: {task['priority']})")

print_result(f"CRM Entries: {len(integration_results['crm_entries'])}", "CRM Integration")
for entry in integration_results['crm_entries']:
    print(f"  - {entry['contact_email']} ({entry['interaction_type']})")

print_result(f"Alerts Generated: {len(integration_results['alerts_generated'])}", "Alert System Integration")
for alert in integration_results['alerts_generated']:
    print(f"  - {alert['subject']} (Type: {alert['type']})")

# Export results as JSON
export_data = {
    'processing_timestamp': datetime.now().isoformat(),
    'emails_processed': len(sample_emails),
    'extraction_results': [{
        'subject': email.subject,
        'sender': email.sender,
        'type': info.email_type,
        'priority': info.priority,
        'sentiment': info.sentiment,
        'action_items_count': len(info.action_items),
        'summary': info.summary
    } for email, info in zip(sample_emails, batch_results)],
    'integration_results': integration_results
}

print_result("Email processing results ready for export", "Export Status")
print(f"Export data structure: {list(export_data.keys())}")

## Conclusion

This notebook demonstrated comprehensive email information extraction using DSPy:

### Key Features Implemented:

1. **Email Classification**: Intelligent categorization of email types and priority levels
2. **Information Extraction**: Extraction of action items, entities, contacts, and deadlines
3. **Sentiment Analysis**: Understanding the tone and emotional context of emails
4. **Automated Summarization**: Concise summaries of email content and key takeaways
5. **Response Generation**: AI-powered suggestions for email responses
6. **Follow-up Tracking**: Identification of emails requiring follow-up actions
7. **Batch Processing**: Efficient processing of multiple emails
8. **System Integration**: Examples of integrating with task management, CRM, and alert systems

### DSPy Features Utilized:

- **Signatures**: Structured definitions for various email processing tasks
- **ChainOfThought**: Step-by-step reasoning for complex analysis
- **Modules**: Modular architecture for different processing components
- **Batch Processing**: Efficient handling of multiple emails

### Best Practices Demonstrated:

1. **Preprocessing**: Cleaning and formatting email content for analysis
2. **Entity Recognition**: Combining AI extraction with regex patterns
3. **Error Handling**: Robust error handling for email processing
4. **Modular Design**: Separating different analysis tasks into focused modules
5. **Integration Patterns**: Examples of connecting with external systems
6. **Reporting**: Comprehensive reporting and analytics

### Use Cases Covered:

- **Customer Support**: Automated triage and response suggestions
- **Project Management**: Action item extraction and task creation
- **Sales and CRM**: Customer communication tracking and analysis
- **Executive Assistance**: Email summarization and priority management
- **Compliance**: Entity extraction and record keeping

### Next Steps:

- Add email attachment processing
- Implement real-time email monitoring
- Add multi-language support
- Create email threading and conversation tracking
- Implement machine learning for continuous improvement
- Add integration with popular email platforms (Gmail, Outlook)

This example showcases how DSPy can be used to build sophisticated email processing systems that understand context, extract meaningful information, and integrate seamlessly with business workflows.